In [1]:
# Standard Libraries
import os
import sys
import datetime
from datetime import timedelta

# Data manipulation and numerical operations
import pandas as pd
import numpy as np

# Machine Learning and Model Evaluation
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Plotting and Visualization
import matplotlib.pyplot as plt

# Custom Functions
sys.path.append('/projectnb/peaclab-mon/boztop/resource-allocation/config')
from nrel_eagle_data_preprocessing import preprocess_data
from ml_model_training import train_model_per_cluster


# Data Preprocessing

In [2]:
file_path = "/projectnb/peaclab-mon/boztop/resource-allocation/datasets/nrel-eagle/eagle_data.csv.bz2"
df_success, df_failure = preprocess_data(file_path)
print("Data preprocessing complete!")
print(f"Number of successful jobs: {len(df_success)}")
print(f"Number of failed jobs: {len(df_failure)}")


Loading data...

Filtering data...

Calculating wait time for successful jobs...



/projectnb/peaclab-mon/boztop/resource-allocation/config/nrel_eagle_data_preprocessing.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['start_time'] = pd.to_datetime(df['start_time'])
/projectnb/peaclab-mon/boztop/resource-allocation/config/nrel_eagle_data_preprocessing.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['submit_time'] = pd.to_datetime(df['submit_time'])
/projectnb/peaclab-mon/boztop/resource-allocation/config/nrel_eagle_data_preprocessing.py:42: SettingWithCopyWarning: 
A value

Encoding categorical features...

Encoding the feature job_id


/projectnb/peaclab-mon/boztop/resource-allocation/config/nrel_eagle_data_preprocessing.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col].astype(str))


Encoding the feature user
Encoding the feature account
Encoding the feature partition
Encoding the feature qos
Encoding the feature name
Encoding the feature work_dir
Encoding the feature submit_line
Preprocessing complete!

Data preprocessing complete!
Number of successful jobs: 7390586
Number of failed jobs: 803002


# Menear et al. 

In [5]:
#sourced from: https://github.com/NREL/eagle-jobs/blob/master/python_scripts/EVAL_FinalModel.py
def baseline_training(train_df, test_df, train_features, target_feature, filename):
    
    train_df = train_df.dropna()
    test_df = test_df.dropna()
    
    X_train = train_df[train_features]
    X_test = test_df[train_features]
    y_train = train_df[target_feature]
    y_test = test_df[target_feature]


    params = { # From Optuna HPO
        'n_estimators': 168,
        'max_depth': 7,
        'learning_rate': 0.3968571956999504,
        'gamma': 0.640232768439118,
        'subsample': 0.747747407403972,
        'colsample_bytree': 0.6280085182287491
    }

    model = XGBRegressor(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    
    pred_vs_act_df = pd.DataFrame(columns=['runtime_pred', 'runtime_act'])
    pred_vs_act_df = pd.DataFrame({'runtime_pred': y_pred, 'runtime_act': y_test})

    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    print(f'r2: {r2:.3f}, rmse: {rmse:.0f}')

    # save the results of the baseline    
    pred_vs_act_df.to_pickle(filename)
        

# Execution Time Prediction Experiments

In [6]:
df = df_success.copy()
df['end_time'] = pd.to_datetime(df['end_time'])
df['end_time']


12         2018-11-14 13:27:34
13         2018-11-14 13:27:38
14         2018-11-14 13:27:36
15         2018-11-14 13:28:05
16         2018-11-14 13:28:05
                   ...        
11030352   2023-02-21 23:01:41
11030353   2023-02-21 22:49:59
11030354   2023-02-21 22:53:52
11030355   2023-02-21 23:01:59
11030362   2023-02-22 00:58:30
Name: end_time, Length: 7390586, dtype: datetime64[ns]

In [13]:
start_dates = pd.date_range(start='2018-11-14', end='2022-08-14', freq='1M')
end_dates = start_dates + pd.DateOffset(months=6)

date_pairs = list(zip(start_dates, end_dates))

train_durations = [20, 40, 60, 80, 100, 120, 140, 160] 



bias_types = ['mean', 'mad', 'std_dev', 'two_sigma']

train_features = ['wallclock_req','processors_req','mem_req','nodes_req','gpus_req','user','partition','name','account']
target_feature = 'run_time'    
req_feature = 'wallclock_req'    
    

In [14]:
def create_sub_dataframes(df, selected_features, n_clusters):

    feature_data = df[selected_features].replace([np.inf, -np.inf], np.nan).fillna(df.mean(numeric_only=True))
    feature_data_scaled = feature_data.values  

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    
    df['cluster'] = kmeans.fit_predict(feature_data_scaled)
    
    cluster_centers = kmeans.cluster_centers_
    
    sub_dataframes = [df[df['cluster'] == cluster].drop(columns=['cluster']) for cluster in range(n_clusters)]
    
    return sub_dataframes, cluster_centers



In [15]:
def make_predictions(test_df, train_features, target_feature, cluster_centers, models, biases, bias_type, filename, add_bias=False):

    test_features = test_df[train_features].values
    distances = np.linalg.norm(cluster_centers[:, np.newaxis] - test_features, axis=2)
    
    test_df.loc[:, 'cluster'] = np.argmin(distances, axis=0)

    pred_vs_act_df = pd.DataFrame(columns=['runtime_pred', 'runtime_act', 'original_index'])

    for cluster_id, (model, bias_dict) in enumerate(zip(models, biases)):
        cluster_data = test_df[test_df['cluster'] == cluster_id]
        if not cluster_data.empty:
            X_test = cluster_data[train_features].values
            y_test = cluster_data[target_feature].values

            if add_bias:
                # Select the appropriate bias based on the specified bias type
                bias = bias_dict.get(bias_type, 0) 
                y_pred = model.predict(X_test) + bias
            else:
                y_pred = model.predict(X_test)

            new_df = pd.DataFrame({
                'runtime_pred': y_pred,
                'runtime_act': y_test,
                'original_index': cluster_data.index
            })
            pred_vs_act_df = pd.concat([pred_vs_act_df, new_df], ignore_index=True)

    # Sort results by the original index
    pred_vs_act_df = pred_vs_act_df.sort_values(by='original_index').reset_index(drop=True)

    # Save results to a file
    pred_vs_act_df.to_pickle(filename)


In [ ]:
import time

for start_date, end_date in date_pairs:
    print(f"\n=== Starting new workflow for Start Date: {start_date.date()} ===\n")
    start_time = time.time()  # Track start time for this configuration

    for train_days in train_durations:
        print(f"[INFO] Processing: Start Date {start_date.date()}, Train {train_days} days")
                
        # Train split
        train_duration = timedelta(days=train_days)
        mid_date = start_date + train_duration
        train_df = df[(df['end_time'] >= start_date) & (df['end_time'] < mid_date)]

        # Test split
        test_df = df[(df['end_time'] >= mid_date) & (df['end_time'] < end_date)]

        # CLUSTERING
        print("[INFO] Creating sub-dataframes and clustering...")
        # Optimal number for clusters is selected to be 12 for the Eagle dataset
        sub_dataframes, cluster_centers = create_sub_dataframes(
            df=train_df, 
            selected_features=train_features, 
            n_clusters=12
        )
        print("[INFO] Clustering completed.")
        
        # Train plain models (XGBoost and RandomForest)
        print("[INFO] Training plain models (XGBoost and RandomForest)...")
        xgb_plain_models, plain_biases = train_model_per_cluster(
            sub_dataframes, train_features, target_feature, 'xgboost', alpha=1)
        rf_plain_models, plain_biases = train_model_per_cluster(
            sub_dataframes, train_features, target_feature, 'rf', alpha=1)
        print("[INFO] Plain models training completed.")


        # BASELINE
        print("[INFO] Running baseline training...")
        baseline_training(train_df, test_df, 
                          train_features, target_feature, 
                          f'eagle_start{start_date}_train{train_days}d_baseline_execution_time_pred_vs_act_df.pkl')
        print("[INFO] Baseline training completed.")


        # Plain XGBoost Predictions
        print("[INFO] Making plain XGBoost predictions...")
        make_predictions(
            test_df, train_features, target_feature, cluster_centers, 
            xgb_plain_models, plain_biases, 'mad', 
            f'eagle_start{start_date}_train{train_days}d_CP_xgboost_execution_time_pred_vs_act_df.pkl', 
            add_bias=False
        )

        # Plain RandomForest Predictions
        print("[INFO] Making plain RandomForest predictions...")
        make_predictions(
            test_df, train_features, target_feature, cluster_centers, 
            rf_plain_models, plain_biases, 'mad', 
            f'eagle_start{start_date}_train{train_days}d_CP_rf_execution_time_pred_vs_act_df.pkl', 
            add_bias=False
        )

        # Bias type evaluations
        for bias_type in bias_types:
            print(f"[INFO] Evaluating with bias type: {bias_type}...")

            # Buffer XGBoost
            make_predictions(
                test_df, train_features, target_feature, cluster_centers,
                xgb_plain_models, plain_biases, bias_type,
                f'eagle_start{start_date}_train{train_days}d_CB_xgboost_{bias_type}_execution_time_pred_vs_act_df.pkl', 
                add_bias=True
            )

            # Buffer RandomForest
            make_predictions(
                test_df, train_features, target_feature, cluster_centers,
                rf_plain_models, plain_biases, bias_type,
                f'eagle_start{start_date}_train{train_days}d_CB_rf_{bias_type}_execution_time_pred_vs_act_df.pkl', 
                add_bias=True
            )


        # USER REQUESTED
        y_test = test_df[target_feature]
        req_test = test_df[req_feature]

        user_req_df = pd.DataFrame({'runtime_pred': req_test, 'runtime_act': y_test})
        filename = f'eagle_start{start_date}_train{train_days}d_user_req_execution_time_pred_vs_act_df.pkl'
        user_req_df.to_pickle(filename)


        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f"[INFO] Completed: Start Date {start_date.date()}, Train {train_days} days - Time Elapsed: {elapsed_time:.2f} seconds\n")

            

        '''
        # Train oversampled models (XGBoost and RandomForest)
        print("[INFO] Training oversampled models (XGBoost and RandomForest)...")
        xgb_oversampled_models, dont_use_biases = train_model_per_cluster(
            sub_dataframes, train_features, target_feature, 'xgboost', alpha=1, stratification=True)
        rf_oversampled_models, dont_use_biases = train_model_per_cluster(
            sub_dataframes, train_features, target_feature, 'xgboost', alpha=1, stratification=True)
        print("[INFO] Oversampled models training completed.")

        
        '''
  

        '''
        # Oversampled XGBoost Predictions
        print("[INFO] Making oversampled XGBoost predictions...")
        make_predictions(
            test_df, train_features, target_feature, cluster_centers, 
            xgb_oversampled_models, dont_use_biases, 'mad',
            f'eagle_train{train_days}d_test{test_days}d_CS_xgboost_execution_time_pred_vs_act_df.pkl', 
            add_bias=False
        )

        # Oversampled RandomForest Predictions
        print("[INFO] Making oversampled RandomForest predictions...")
        make_predictions(
            test_df, train_features, target_feature, cluster_centers, 
            rf_oversampled_models, dont_use_biases, 'mad',
            f'eagle_train{train_days}d_test{test_days}d_CS_rf_execution_time_pred_vs_act_df.pkl', 
            add_bias=False
        )

        '''


            


=== Starting new workflow for Start Date: 2018-11-30 ===

[INFO] Processing: Start Date 2018-11-30, Train 20 days
[INFO] Creating sub-dataframes and clustering...
[INFO] Clustering completed.
[INFO] Training plain models (XGBoost and RandomForest)...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[INFO] Plain models training completed.
[INFO] Running baseline training...
r2: -0.135, rmse: 33046
[INFO] Baseline training completed.
[INFO] Making plain XGBoost predictions...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


[INFO] Making plain RandomForest predictions...


...

[INFO] Evaluating with bias type: two_sigma...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


[INFO] Completed: Start Date 2022-04-30, Train 100 days - Time Elapsed: 196.72 seconds

[INFO] Processing: Start Date 2022-04-30, Train 120 days
[INFO] Creating sub-dataframes and clustering...


/project/peaclab-mon/monitoring_venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


[INFO] Clustering completed.
[INFO] Training plain models (XGBoost and RandomForest)...


# Some Visuals

In [ ]:
# LET'S VISUALIZE THE OVERPREDICTION FOR THE EXECUTION TIME
plt.figure(figsize=(10, 6))

# Ensure the columns are numeric
df['wallclock_req'] = pd.to_numeric(df['wallclock_req'], errors='coerce')
df['run_time'] = pd.to_numeric(df['run_time'], errors='coerce')

random_indices = np.random.choice(df.index, size=150, replace=False)
random_indices_sorted = np.sort(random_indices)

plt.scatter(df['wallclock_req'].iloc[random_indices_sorted].index, 
            df['wallclock_req'].iloc[random_indices_sorted], 
            label='User Requested Execution Time', marker='o', color='#FF4500')

plt.scatter(df['run_time'].iloc[random_indices_sorted].index, 
            df['run_time'].iloc[random_indices_sorted], 
            label='Actual Execution Time', marker='x', color='purple')

plt.title('Comparing the Execution Time Requests with Actual Results (NREL Eagle Dataset)')
plt.xlabel('Job Index')
plt.ylabel('Seconds')

plt.legend()

save_path = '/projectnb/peaclab-mon/boztop/resource-allocation/plots/nrel_eagle_exec_time.png'  # Change this to your desired path
plt.savefig(save_path)

plt.show()


In [ ]:
# Demonstrating the queue wait time increases with increasing wait time

bins = [0, 0.1, 1, 10, 100, float('inf')]
labels = ['[0, 0.1]', '(0.1, 1]', '(1, 10]', '(10, 100]', '(100, ∞)']

df_success['wallclock_req_hour'] = df_success['wallclock_req'] / 3600
df_success['wait_time_hour'] = df_success['wait_time'] / 3600


df_success['wallclock_category'] = pd.cut(df_success['wallclock_req_hour'], bins=bins, labels=labels)

wait_time_means = df_success.groupby('wallclock_category')['wait_time_hour'].mean()

plt.figure(figsize=(10, 6))
wait_time_means.plot(kind='bar', color='green', edgecolor='black')

# Adding labels and title
plt.xlabel('Requested Wallclock Time (hours)', fontsize=20)
plt.ylabel('Average Wait Time (hours)',fontsize=20)
plt.title('NREL Eagle: Average Wait Time by Requested Wallclock Time',fontsize=20)
plt.yticks(fontsize=14)
plt.grid(True)

# Show the plot
plt.xticks(rotation=45)
plt.xticks(fontsize=16)
plt.show()




# Find Optimal Feature Set

In [ ]:
df_success.columns

In [ ]:
for col in ['job_id', 'user', 'account', 'partition', 'qos', 'name', 'work_dir','submit_line']:
    print(f'Encoding the feature {col}')
    le = LabelEncoder()
    df_success[col] = le.fit_transform(df_success[col].astype(str))
    
    

In [ ]:
def find_optimal_feature_set(features, pred_feature, title, xlabel):
    X = df_success[features]
    y = df_success[pred_feature]
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rmse_values = []
    combinations = []
    
    for r in range(5,7):
        print(f'Iteration: {r}')
        for combo in itertools.combinations(features, r):
            print(f'Feature Combination: {list(combo)}')
            X_train_combo = X_train[list(combo)]
            X_test_combo = X_test[list(combo)]

            model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='rmse')
            model.fit(X_train_combo, y_train)

            y_pred = model.predict(X_test_combo)

            rmse = np.sqrt(mean_squared_error(y_test, y_pred))

            rmse_values.append(rmse)
            combinations.append(', '.join(combo))
        
    sorted_indices = np.argsort(rmse_values)[::-1]
    sorted_rmse_values = np.array(rmse_values)[sorted_indices]
    sorted_combinations = np.array(combinations)[sorted_indices]

    norm = plt.Normalize(vmin=min(sorted_rmse_values), vmax=max(sorted_rmse_values))
    colors = cm.viridis(norm(sorted_rmse_values)) 

    plt.figure(figsize=(10, 6))
    bars = plt.barh(sorted_combinations, sorted_rmse_values, color=colors)

    cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap='viridis'))
    cbar.set_label('RMSE', fontsize=12)

    plt.xlabel(f'{xlabel}', fontsize=14)
    plt.ylabel('Feature Combination', fontsize=14)
    plt.title(f'{title}', fontsize=16)
    plt.tight_layout()
    plt.show()

    
    

In [ ]:
'''
find_optimal_feature_set(['job_id', 'user', 'account', 'partition', 'qos', 'wallclock_req',
       'nodes_req', 'processors_req', 'gpus_req', 'mem_req','name', 'work_dir',
       'submit_line', 'wait_time', 'wallclock_req_hour', 'wait_time_hour'], 'run_time', 'RMSE Values for Each Input Feature Combination', 'RMSE (sec)')
       
'''



# Execution Time Prediction

In [ ]:
rmse = np.sqrt(((df_success['run_time'] - df_success['wallclock_req']) ** 2).mean())

print(f"RMSE: {rmse}")

In [ ]:
rmse, mae, r2, y_test, y_pred, req_test = train_xgboost(df_success, 'NREL Eagle Dataset', 'run_time', ['job_id', 'user', 'account', 'partition', 'qos', 'wallclock_req',
       'nodes_req', 'processors_req', 'gpus_req', 'mem_req', 'name', 'work_dir',
       'submit_line', 'wait_time', 'wallclock_req_hour', 'wait_time_hour' ] , 'wallclock_req',)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R^2: {r2:.4f}")


In [ ]:
rmse, mae, r2, y_test, y_pred, req_test = train_xgboost(df_success, 'NREL Eagle Dataset', 'run_time', ['wallclock_req','mem_req','processors_req','user','partition'] , 'wallclock_req',)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R^2: {r2:.4f}")


In [ ]:
sub_dfs = cluster_create_sub_dataframes(df_success, ['job_id', 'user', 'account', 'partition', 'qos', 'wallclock_req',
       'nodes_req', 'processors_req', 'gpus_req', 'mem_req', 'name', 'work_dir',
       'submit_line', 'wait_time', 'wallclock_req_hour', 'wait_time_hour'] , ['run_time'],10)

rmse_list = []
mae_list = []
r2_list = []

metrics_data = []
overprediction_sums = []

for i, sub_df in enumerate(sub_dfs):
    print(f"Processing cluster {i+1}/{len(sub_dfs)}")
    
    rmse, mae, r2, y_test, y_pred, req_test= train_xgboost(sub_df, 'NREL Eagle Dataset', 'run_time', ['wallclock_req','mem_req','processors_req','user','partition'], 'wallclock_req')
    
    if rmse is not None:
        rmse_list.append(rmse)
        mae_list.append(mae)
        r2_list.append(r2)

        total_jobs = len(y_test)
        requested_gt_target = (req_test > y_test).sum()
        predicted_lt_actual = (y_pred < y_test).sum()
        predicted_gt_actual = (y_pred > y_test).sum()

        op1 = y_pred > y_test
        op2 = req_test > y_test
        
        sum_predicted_minus_actual = (y_pred[op1] - y_test[op1]).sum()
        sum_requested_minus_actual = (req_test[op2] - y_test[op2]).sum()

        metrics_data.append([i + 1, total_jobs, requested_gt_target, predicted_lt_actual, predicted_gt_actual])
        overprediction_sums.append([sum_predicted_minus_actual, sum_requested_minus_actual])

        


rmse = np.mean(rmse_list)
print(f"Average RMSE for Clustering: {rmse:.4f}")
mae = np.mean(mae_list)
print(f"Average MAE for Clustering: {mae:.4f}")
r2 = np.mean(r2_list)
print(f"Average R^2 for Clustering: {r2:.4f}")


plot_everything(metrics_data, overprediction_sums, 
                'Execution Time Experiment on NREL Eagle Dataset: Over and Under Prediction Analysis ', 
                'NREL Eagle Dataset: Sum of Differences in Overprediction Cases Across Clusters',
                'Execution Time (seconds)')



# User Variance Study

In [ ]:
zero_runtime_count = df_success[df_success['run_time'] == 0].shape[0]

# Print the result
print(f"Number of jobs with run_time == 0: {zero_runtime_count}")

In [ ]:
user_stats = df_success.groupby('user')['run_time'].agg(['count', 'mean', 'std']).reset_index()
user_stats = user_stats.rename(columns={'count': 'job_count', 'mean': 'avg_runtime', 'std': 'runtime_std'})

top_20_users = user_stats.nlargest(20, 'job_count')

plt.figure(figsize=(12, 6))
plt.bar(top_20_users['user'], top_20_users['job_count'], yerr=top_20_users['runtime_std'], capsize=5)
plt.xticks(rotation=45, ha='right')
plt.xlabel('User')
plt.ylabel('Number of Jobs Submitted')
plt.title('Top 20 Users by Job Submissions with Runtime Variation')
plt.tight_layout()

plt.show()


In [ ]:
user_stats = df_success.groupby('user')['run_time'].agg(job_count='count', min_runtime='min', max_runtime='max').reset_index()

top_20_users = user_stats.nlargest(20, 'job_count')

plt.figure(figsize=(12, 6))
plt.errorbar(
    top_20_users['user'], 
    (top_20_users['min_runtime'] + top_20_users['max_runtime']) / 2,  # Mid-point of min and max for plotting
    yerr=[top_20_users['max_runtime'] - (top_20_users['min_runtime'] + top_20_users['max_runtime']) / 2, 
          (top_20_users['min_runtime'] + top_20_users['max_runtime']) / 2 - top_20_users['min_runtime']],
    fmt='o', capsize=5, color='blue'
)

# Step 4: Add job count as text labels on top of each error bar
for i, (user, job_count) in enumerate(zip(top_20_users['user'], top_20_users['job_count'])):
    plt.text(i, top_20_users['max_runtime'].iloc[i] + 1, str(job_count), ha='center', va='bottom', fontsize=10)

# Customize plot
plt.xticks(rotation=45, ha='right')
plt.xlabel('User')
plt.ylabel('Runtime (min-max)')
plt.title('Top 20 Users by Job Submissions with Min-Max Runtime and Job Counts')
plt.tight_layout()

plt.show()


In [ ]:

# Step 1: Calculate job count, min, max, and variance of runtime for each user
user_stats = df_success.groupby('user')['run_time'].agg(
    job_count='count', min_runtime='min', max_runtime='max', runtime_variance='var'
).reset_index()

# Step 2: Select the top 20 users by runtime variance
top_20_users = user_stats.nlargest(20, 'runtime_variance')

# Step 3: Plot min-max runtime as error bars with job counts as labels
plt.figure(figsize=(12, 6))
plt.errorbar(
    top_20_users['user'], 
    (top_20_users['min_runtime'] + top_20_users['max_runtime']) / 2,  # Mid-point of min and max for plotting
    yerr=[top_20_users['max_runtime'] - (top_20_users['min_runtime'] + top_20_users['max_runtime']) / 2, 
          (top_20_users['min_runtime'] + top_20_users['max_runtime']) / 2 - top_20_users['min_runtime']],
    fmt='o', capsize=5, color='blue'
)

# Step 4: Add job count as text labels on top of each error bar
for i, (user, job_count) in enumerate(zip(top_20_users['user'], top_20_users['job_count'])):
    plt.text(i, top_20_users['max_runtime'].iloc[i] + 1, str(job_count), ha='center', va='bottom', fontsize=10)

# Customize plot
plt.xticks(rotation=45, ha='right')
plt.xlabel('User')
plt.ylabel('Runtime (min-max)')
plt.title('Top 20 Users by Runtime Variation with Min-Max Runtime and Job Counts')
plt.tight_layout()

plt.show()


In [ ]:
def calculate_variance(df, target_column):
    stats = df.groupby('user')[target_column].var().reset_index(name='Variance')
    return stats

df_filtered = df_success[df_success['run_time'] != 0]

job_counts = df_filtered.groupby('user').size().reset_index(name='job_count')

bins = [0, 5, 20, 50, 100, 250, 500, 1000, 2500, job_counts['job_count'].max()]
labels = ['1-5', '6-20', '21-50', '51-100', '101-250', '251-500', '501-1000', '1001-2500', '2500+']
job_counts['job_range'] = pd.cut(job_counts['job_count'], bins=bins, labels=labels, right=False)

variance_stats = calculate_variance(df_filtered, 'run_time')

job_counts = job_counts.merge(variance_stats, on='user')

job_counts = job_counts.dropna(subset=['Variance'])

range_distribution = job_counts['job_range'].value_counts(normalize=True).sort_index() * 100

avg_variance_by_range = job_counts.groupby('job_range')['Variance'].mean()

# Plot the bar plot
plt.figure(figsize=(12, 6))
bars = plt.bar(range_distribution.index, range_distribution.values, color='purple')
plt.xlabel('Number of Jobs Submitted (Range)')
plt.ylabel('Percentage of Users (%)')
plt.title('Percentage of Users by Job Submission Range')

# Add percentage labels on top of bars
for bar, pct in zip(bars, range_distribution.values):
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{pct:.1f}%', 
             ha='center', va='bottom', fontsize=10, color='black')

# Display average variance values above each bar
for i, bar in enumerate(bars):
    avg_variance = avg_variance_by_range.iloc[i]
    # Format the variance with 3 significant figures in scientific notation
    formatted_variance = f'{avg_variance:.3e}'
    plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1, 
             f'Avg Var: {formatted_variance}', ha='center', va='bottom', fontsize=10, color='blue')

# Display the plot
plt.tight_layout()
plt.show()

# Print the average variance by job submission range
print("Average run_time variance by job submission range:")
print(avg_variance_by_range)


# Execution time variation for users

In [ ]:
def calculate_user_variation(df, user, target_columns, plot_type='scatter'):
    user_df = df[df['user'] == user]
    
    std_devs = {}
    
    for target_column in target_columns:
        std_dev = user_df[target_column].std()
        std_devs[target_column] = std_dev
        print(f"Standard deviation for '{target_column}' for user '{user}': {std_dev}")
    
    plt.figure(figsize=(10, 6))
    
    for target_column in target_columns:
        if plot_type == 'line':
            plt.plot(user_df[target_column], marker='o', linestyle='-', label=target_column)
        elif plot_type == 'scatter':
            plt.scatter(user_df.index, user_df[target_column], label=target_column)
    
    plt.title(f"Variation of Requested and Actual Runtime for User '{user}'")
    plt.xlabel("Index")
    plt.ylabel("Time Values (seconds)")
    plt.legend()
    plt.show()
    
    return std_devs

calculate_user_variation(df_success,'user0314', ['wallclock_req', 'run_time'])


In [ ]:
def calculate_all_users_variation(df, target_column):
    # Step 1: Exclude users with only one entry
    stats = df.groupby('user')[target_column].agg(['std', 'var', 'count']).reset_index()
    stats_filtered = stats[stats['count'] > 1]  # Exclude users with only one entry
    
    if stats_filtered.shape[0] != stats.shape[0]:
        excluded_users = stats[stats['count'] == 1]
        print(f"Users with only one entry (excluded from calculations): {len(excluded_users)}")
        print(excluded_users[['user', 'count']])
    
    # Step 2: Merge the variance statistics back into the original dataframe
    df_with_variance = df.merge(stats_filtered[['user', 'var', 'std']], on='user', how='left')

    # Step 3: Split users into two categories based on variance (5% range)
    variance_5_percent_threshold = stats_filtered['var'].quantile(0.05)
    variance_5_percent_df = df_with_variance[df_with_variance['var'] <= variance_5_percent_threshold]
    remaining_users_df = df_with_variance[df_with_variance['var'] > variance_5_percent_threshold]
    
    # Step 4: Quantile binning for variance range (for visualization)
    quantile_bins = pd.qcut(stats_filtered['var'], q=10, duplicates='drop')
    stats_filtered['variance_range'] = quantile_bins
    
    # Step 5: Distribution of users in each variance range
    range_distribution = stats_filtered['variance_range'].value_counts().sort_index()
    
    # Plot the distribution of users by variance range
    plt.figure(figsize=(12, 6))
    bars = plt.bar(range_distribution.index.astype(str), range_distribution.values, color='teal')
    plt.xlabel('Variance Range')
    plt.ylabel('Number of Users')
    plt.title('Distribution of Users by Run Time Variance Range')

    for bar, count in zip(bars, range_distribution.values):
        percentage = f"{(count / len(stats_filtered) * 100):.1f}%"
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{count}\n({percentage})', 
                 ha='center', va='bottom', fontsize=10, color='black')

    plt.xticks(rotation=45)  # Rotate labels for readability
    plt.tight_layout()
    plt.show()

    # Return the full dataframes with all features for both categories
    return variance_5_percent_df, remaining_users_df



In [ ]:
variance_5_percent_df, remaining_users_df = calculate_all_users_variation(df_success, 'run_time')



In [ ]:
def calculate_user_variation_by_percentile(df, target_column):
    stats = df.groupby('user')[target_column].agg(['std', 'var', 'count']).reset_index()
    stats_filtered = stats[stats['count'] > 1] 

    if stats_filtered.shape[0] != stats.shape[0]:
        excluded_users = stats[stats['count'] == 1]
        print(f"Users with only one entry (excluded from calculations): {len(excluded_users)}")

    df_with_variance = df.merge(stats_filtered[['user', 'var', 'std']], on='user', how='left')
    
    percentiles = pd.qcut(stats_filtered['var'], q=20, labels=False)
    stats_filtered['variance_percentile'] = percentiles

    percentile_dataframes = []
    for percentile in range(20):
        percentile_df = df_with_variance[df_with_variance['user'].isin(
            stats_filtered[stats_filtered['variance_percentile'] == percentile]['user']
        )].drop(columns=['var', 'std'])  # Drop 'var' and 'std'
        percentile_dataframes.append(percentile_df)

    range_distribution = stats_filtered['variance_percentile'].value_counts().sort_index()
    plt.figure(figsize=(12, 6))
    bars = plt.bar(range_distribution.index, range_distribution.values, color='teal')
    plt.xlabel('Variance Percentile')
    plt.ylabel('Number of Users')
    plt.title('Distribution of Users by Run Time Variance Percentile')

    for bar, count in zip(bars, range_distribution.values):
        percentage = f"{(count / len(stats_filtered) * 100):.1f}%"
        plt.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{count}\n({percentage})', 
                 ha='center', va='bottom', fontsize=10, color='black')

    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return percentile_dataframes


In [ ]:
percentile_dfs = calculate_user_variation_by_percentile(df_success, 'run_time')

In [ ]:
'''
# The time columns are of the ISO 8601 format
df['wallclock_used_seconds'] = df['wallclock_used'].apply(isodate.parse_duration).dt.total_seconds()
df['wallclock_req_seconds'] = df['wallclock_req'].apply(isodate.parse_duration).dt.total_seconds()
df['cpu_used_seconds'] = df['cpu_used'].apply(isodate.parse_duration).dt.total_seconds()
df['queue_wait_seconds'] = df['queue_wait'].apply(isodate.parse_duration).dt.total_seconds()
'''
# The time columns are of datetime format

# Showing correlations wrt user run_time variances

In [ ]:
def process_and_plot_correlation(df, target_column):
    label_encoders = {}
    cols_to_encode = ['job_id', 'user', 'account', 'partition', 'qos', 'name', 'work_dir']
    
    for col in cols_to_encode:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le
    
    correlation_matrix = df.corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title(f'Correlation Matrix of {target_column} and Other Columns')
    plt.show()
    
    return df, label_encoders

In [ ]:
def calculate_positive_correlations(df, target_column, cols_to_encode=['job_id', 'user', 'account', 'partition', 'qos', 'name', 'work_dir']):
    # Encode categorical columns
    label_encoders = {}
    for col in cols_to_encode:
        if col in df.columns:  # Ensure the column exists in the current dataframe
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            label_encoders[col] = le
    
    # Calculate correlation matrix
    correlation_matrix = df.corr()
    
    # Plot correlation matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title(f'Correlation Matrix of {target_column} and Other Columns')
    plt.show()
    
    # Identify positively correlated features with the target column
    positive_corr_features = correlation_matrix[target_column][correlation_matrix[target_column] > 0].index.tolist()
    positive_corr_features.remove(target_column)  # Remove the target column itself from the list
    
    return positive_corr_features, label_encoders

In [ ]:
processed_df, encoders = process_and_plot_correlation(variance_5_percent_df, 'run_time')

In [ ]:
processed_df, encoders = process_and_plot_correlation(remaining_users_df, 'run_time')

In [ ]:
df_success_encoded, _ = process_and_plot_correlation(df_success, 'run_time')

In [ ]:
def process_all_dataframes(dataframes, target_column):
    positive_correlations_dict = {}
    all_label_encoders = {}

    for i, df in enumerate(dataframes):
        # Process each dataframe and calculate positive correlations
        positive_corr_features, label_encoders = calculate_positive_correlations(df, target_column)
        
        # Store results for each percentile dataframe
        positive_correlations_dict[f'Percentile_{i + 1}'] = positive_corr_features
        all_label_encoders[f'Percentile_{i + 1}'] = label_encoders
    
    return positive_correlations_dict, all_label_encoders


In [ ]:
positive_correlations_dict, all_label_encoders = process_all_dataframes(percentile_dfs, 'run_time')

# 2019 dataset operations

In [ ]:
import re
import pandas as pd
import bz2

In [ ]:
def iso_duration_to_seconds(duration_str):
    match = re.match(r'P(?P<days>\d+)DT(?P<hours>\d+)H(?P<minutes>\d+)M(?P<seconds>\d+)S', duration_str)
    if match:
        days = int(match.group('days'))
        hours = int(match.group('hours'))
        minutes = int(match.group('minutes'))
        seconds = int(match.group('seconds'))

        total_seconds = days * 86400 + hours * 3600 + minutes * 60 + seconds
        return total_seconds
    else:
        raise ValueError("Invalid ISO 8601 duration format")



In [ ]:
df_json = pd.read_json('/projectnb/peaclab-mon/boztop/resource-allocation/datasets/nrel-eagle/eagle_jobs_2019-12.json.bz2', orient='records', compression='bz2', lines=True)

df_json


In [ ]:
# ****** Columns in the json file and feature explanations: ******

# wallclock_used: The total wall-clock time used by the job in ISO 8601 duration format (e.g., "P2DT0H0M8S" means 2 days and 8 seconds).
# nodelist: A list of nodes allocated for the job (e.g., ["r8i0n11", "r8i0n15"]).
# user: The user ID who submitted the job, represented as a unique identifier (e.g., "FB976BF71E5B").
# partition: The partition (or queue) where the job was submitted (e.g., "standard").
# cpu_used: The total CPU time used by the job in ISO 8601 duration format (e.g., "P576DT0H38M24S").
# wallclock_req: The wall-clock time requested for the job in ISO 8601 duration format (e.g., "P2DT0H0M0S").
# submit_time: The time when the job was submitted in ISO 8601 format (e.g., "2020-08-31T05:20:50.000Z").
# processors_req: The number of processors (CPUs) requested for the job (e.g., 288).
# start_time: The time when the job started execution (e.g., "2020-08-31T05:20:54.000Z").
# std_power: The standard power consumption for the job (e.g., 10.025973 watts).
# nodes_req: The number of nodes requested by the job (e.g., 8).
# queue_wait: The total time the job spent waiting in the queue before starting, in ISO 8601 duration format (e.g., "P0DT0H0M4S").
# state: The final state of the job (e.g., "TIMEOUT" for jobs that timed out).
# qos: The Quality of Service (QoS) level for the job, which may affect priority (e.g., "standby").
# script: A unique identifier for the job's submission script (e.g., "A539757FAAEE").
# account: The account under which the job was charged (e.g., "BD8618B13179").
# nodes_used: The number of nodes actually used by the job during execution (e.g., 8).
# end_time: The time when the job ended (e.g., "2020-09-02T05:21:02.000Z").
# job_id: A unique identifier for the job (e.g., 4108812).
# array_pos: The position of the job in an array if it was part of an array job (could be null if not applicable).
# avg_power: The average power consumption by the job (e.g., 394.68008 watts).
# processors_used: The actual number of processors (CPUs) used by the job during execution (e.g., 288).


df_json = df_json.drop(columns = ['array_pos', 'partition', 'submit_time', 'start_time',  'script', 'end_time', 'nodelist'])

df_json['wallclock_used_seconds'] = df_json['wallclock_used'].apply(iso_duration_to_seconds)
df_json['wallclock_req_seconds'] = df_json['wallclock_req'].apply(iso_duration_to_seconds)
df_json['queue_wait_seconds'] = df_json['queue_wait'].apply(iso_duration_to_seconds)
df_json['cpu_seconds'] = df_json['cpu_used'].apply(iso_duration_to_seconds)

df_json = df_json.drop(columns = ['wallclock_used', 'wallclock_req', 'queue_wait', 'cpu_used'])


In [ ]:
df_json

In [ ]:
# Demonstrating the queue wait time increases with increasing wait time

bins = [0, 0.1, 1, 10, 100, float('inf')]
labels = ['[0, 0.1]', '(0.1, 1]', '(1, 10]', '(10, 100]', '(100, ∞)']

df_success['wallclock_req_hour'] = df_success['wallclock_req'] / 3600
df_success['wait_time_hour'] = df_success['wait_time'] / 3600


df_success['wallclock_category'] = pd.cut(df_success['wallclock_req_hour'], bins=bins, labels=labels)

wait_time_means = df_success.groupby('wallclock_category')['wait_time_hour'].mean()

plt.figure(figsize=(10, 6))
wait_time_means.plot(kind='bar', color='green', edgecolor='black')

# Adding labels and title
plt.xlabel('Requested Wallclock Time (hours)', fontsize=14)
plt.ylabel('Average Wait Time (hours)',fontsize=14)
plt.title('NREL Eagle: Average Wait Time by Requested Wallclock Time',fontsize=14)
plt.yticks(fontsize=14)
plt.grid(True)

# Show the plot
plt.xticks(rotation=45)
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_json['user_encoded'] = label_encoder.fit_transform(df_json['user'])
df_json['account_encoded'] = label_encoder.fit_transform(df_json['account'])


In [ ]:
# --- LET'S MAKE SOME OBSERVATIONS! ---
print(f"Total number of jobs in the dataset: {df_json.shape[0]}")

print(f"Number of unique users: {df_json['user_encoded'].nunique()}")
      
print(f"Number of unique state of jobs: {df_json['state'].nunique()}")  
      
print("The unique state of jobs:")  
      
print(df_json['state'].value_counts())
      
print("The unique qos of jobs:")  
      
print(df_json['qos'].value_counts())

    
df_json_success = df_json[df_json['state'] == 'COMPLETED']
df_json_failure = df_json[df_json['state'] == 'FAILED']       

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_correlation_heatmap(df, name):
    plt.figure(figsize=(10, 8))
    corr_matrix = df.corr()
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title(f"Correlation Matrix Heatmap for {name} Jobs in Eagle Dataset")
    plt.show()
     

In [ ]:
plot_correlation_heatmap(df_json_success, 'Successful') 

In [ ]:
plot_correlation_heatmap(df_json_failure, 'Failed') 

In [ ]:
pairs_to_plot = [('nodes_used', 'wallclock_used_seconds'), ('processors_used', 'wallclock_used_seconds'), ('nodes_used', 'processors_used'), ('nodes_req', 'processors_req')]

fig, axes = plt.subplots(1, len(pairs_to_plot), figsize=(12, 5))

for i, (x_col, y_col) in enumerate(pairs_to_plot):
    axes[i].scatter(df_json_success[x_col], df_json_success[y_col], color='b')
    axes[i].set_title(f"{x_col} vs {y_col}")
    axes[i].set_xlabel(x_col)
    axes[i].set_ylabel(y_col)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
# LET'S VISUALIZE THE OVERPREDICTION FOR THE EXECUTION TIME
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10, 6))


#random_indices = np.random.choice(df_json_success.index, size=50, replace=False)
#random_indices_sorted = np.sort(random_indices)

random_indices = np.random.randint(0, len(df_json_success), size=100)
random_indices_sorted = np.sort(random_indices[random_indices < len(df_json_success)])


plt.plot(df_json_success['wallclock_req_seconds'].iloc[random_indices_sorted], label='User Requested Execution Time', marker='o', color='#FF4500')
plt.plot(df_json_success['wallclock_used_seconds'].iloc[random_indices_sorted], label='Actual Execution Time', marker='x', color='purple')


plt.title('Comparing the Execution Time Requests with Actual Results (NREL Eagle Dataset)')
plt.xlabel('Job Index')
plt.ylabel('Seconds')


plt.legend()

#save_path = '/projectnb/peaclab-mon/boztop/resource-allocation/plots/nrel_eagle_exec_time.png'  # Change this to your desired path
#plt.savefig(save_path)

plt.show()

# Clustering

In [ ]:
MAX_PROCESSORS_PER_NODE = 36

df_json_success['min_nodes_needed'] = (df_json_success['processors_req'] / MAX_PROCESSORS_PER_NODE).apply(np.ceil)

# Create a new 'group' column based on the nodes requirement comparison
def categorize_row(row):
    if row['nodes_req'] >= row['min_nodes_needed']:
        return 'sufficient_nodes'
    else:
        return 'insufficient_nodes'

df_json_success['group'] = df_json_success.apply(categorize_row, axis=1)

# Drop the helper column if no longer needed
df_json_success.drop(columns=['min_nodes_needed'], inplace=True)

# Output the grouped data
print(df_json_success['group'].value_counts())
print(df_json_success.head())


In [ ]:
highly_correlated_columns = ['cpu_seconds', 'wallclock_req_seconds','nodes_req', 'processors_req','nodes_used', 'processors_used',] 
df_json_success_subset = df_json_success[highly_correlated_columns]

df_json_success_subset.replace([np.inf, -np.inf], np.nan, inplace=True)
df_json_success_subset.fillna(df_json_success_subset.mean(), inplace=True) 

# Apply KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
df_json_success['cluster'] = kmeans.fit_predict(df_json_success_subset)

# Output the clustered data
print(df_json_success['cluster'].value_counts())
print(df_json_success.head())

# Training Clusters

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os


features = ['wallclock_req_seconds','nodes_req', 'processors_req','nodes_used', 'processors_used', 'cpu_seconds']
target = 'wallclock_used_seconds'

colors = [
    'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white',
    'orange', 'purple', 'brown', 'pink', 'lime', 'teal', 'indigo', 'violet',
    '#FF6347', '#4682B4', '#32CD32', '#FFD700', '#6A5ACD', '#FF4500', '#DA70D6'
]

output_dir = "/projectnb/peaclab-mon/boztop/resource-allocation/plots/"
os.makedirs(output_dir, exist_ok=True)


total_underpredictions = 0
total_overpredictions = 0
total_samples = 0  # To track the total number of samples across clusters


# Loop over each cluster
for cluster_id in df_json_success['cluster'].unique():
    print(f"Training XGBoost for Cluster {cluster_id}")
    
    cluster_data = df_json_success[df_json_success['cluster'] == cluster_id]
    
    X = cluster_data[features]
    y = cluster_data[target]
    wallclock_requests = cluster_data['wallclock_req_seconds']
    
    
    X_train, X_test, y_train, y_test, wallclock_train, wallclock_test = train_test_split(
        X, y, wallclock_requests, test_size=0.2, random_state=42)
    
    
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f"Cluster {cluster_id} - Mean Squared Error: {mse}")
    
    # Calculate underpredictions and overpredictions
    underpredictions = (y_pred < y_test).sum()
    underprediction_percentage = (underpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

    overpredictions = (y_pred > y_test).sum()
    overprediction_percentage = (overpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

    print(f"Cluster {cluster_id} - Number of underpredictions: {underpredictions} ({underprediction_percentage:.2f}%)")
    print(f"Cluster {cluster_id} - Number of overpredictions: {overpredictions} ({overprediction_percentage:.2f}%)")

    # Update totals
    total_underpredictions += underpredictions
    total_overpredictions += overpredictions
    total_samples += len(y_test)
    
    # Scatter plot of y_test vs y_pred
    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, color=np.random.choice(colors), label='Predicted vs Actual')
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Perfect prediction line')
    plt.xlabel('Actual Runtime')
    plt.ylabel('Predicted Runtime')
    plt.title(f'Cluster {cluster_id} - Predicted vs Actual Runtime')
    plt.legend()
    plt.grid(True)
    plot_filename = os.path.join(output_dir, f'nrel_eagle_cluster_{cluster_id}_runtime_prediction.png')
    plt.savefig(plot_filename)
    print(f"Figure saved for Cluster {cluster_id}: {plot_filename}")
    plt.show()

    # Wallclock comparison plot
    plt.figure(figsize=(8, 6))
    improvement = wallclock_test - y_pred  # Improvement = requested wallclock - predicted runtime
    plt.scatter(wallclock_test, improvement, color=np.random.choice(colors), label='Improvement')
    plt.axhline(y=0, color='red', linestyle='--', label='No Improvement')
    plt.xlabel('Requested Wallclock Time')
    plt.ylabel('Improvement (Wallclock - Predicted)')
    plt.title(f'Cluster {cluster_id} - Wallclock Time Improvement')
    plt.legend()
    plt.grid(True)
    plot_filename = os.path.join(output_dir, f'nrel_eagle_cluster_{cluster_id}_runtime_improvement.png')
    plt.savefig(plot_filename)
    print(f"Figure saved for Cluster {cluster_id}: {plot_filename}")
    plt.show()
    
if total_samples > 0:
    total_underprediction_percentage = (total_underpredictions / total_samples) * 100
    total_overprediction_percentage = (total_overpredictions / total_samples) * 100
    print(f"\nTotal Number of underpredictions: {total_underpredictions} ({total_underprediction_percentage:.2f}%)")
    print(f"Total Number of overpredictions: {total_overpredictions} ({total_overprediction_percentage:.2f}%)")


# Training the Whole Dataset

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import os


features = ['wallclock_req_seconds','nodes_req', 'processors_req','nodes_used', 'processors_used', 'cpu_seconds']
target = 'wallclock_used_seconds'


output_dir = "/projectnb/peaclab-mon/boztop/resource-allocation/plots/"
os.makedirs(output_dir, exist_ok=True)

X = df_json_success[features]
y = df_json_success[target]
wallclock_requests = df_json_success['wallclock_req_seconds']
    
    
X_train, X_test, y_train, y_test, wallclock_train, wallclock_test = train_test_split(
    X, y, wallclock_requests, test_size=0.2, random_state=42)
    
    
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"NREL Eagle Total Dataset Mean Squared Error: {mse}")


underpredictions = (y_pred < y_test).sum()
underprediction_percentage = (underpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

overpredictions = (y_pred > y_test).sum()
overprediction_percentage = (overpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

print(f"NREL Eagle Total Dataset Number of underpredictions: {underpredictions} ({underprediction_percentage:.2f}%)")
print(f"NREL Eagle Total Dataset Number of overpredictions: {overpredictions} ({overprediction_percentage:.2f}%)")

    
    
# Scatter plot of y_test vs y_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color=np.random.choice(colors), label='Predicted vs Actual')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Perfect prediction line')
plt.xlabel('Actual Runtime')
plt.ylabel('Predicted Runtime')
plt.title(f'NREL Eagle Total Dataset Predicted vs Actual Runtime')
plt.legend()
plt.grid(True)
plot_filename = os.path.join(output_dir, f'nrel_eagle_all_data_runtime_prediction.png')
plt.savefig(plot_filename)
plt.show()

# Wallclock comparison plot
plt.figure(figsize=(8, 6))
improvement = wallclock_test - y_pred  # Improvement = requested wallclock - predicted runtime
plt.scatter(wallclock_test, improvement, color=np.random.choice(colors), label='Improvement')
plt.axhline(y=0, color='red', linestyle='--', label='No Improvement')
plt.xlabel('Requested Wallclock Time')
plt.ylabel('Improvement (Wallclock - Predicted)')
plt.title(f'NREL Eagle Total Dataset Wallclock Time Improvement')
plt.legend()
plt.grid(True)
plot_filename = os.path.join(output_dir, f'nrel_eagle_all_data_runtime_improvement.png')
plt.savefig(plot_filename)
plt.show()
    

In [ ]:
# 3. Visualize Clusters in 2D with PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df_json_success.drop(columns=['kmeans_cluster']))
df_json_success['pca_one'] = pca_result[:, 0]
df_json_success['pca_two'] = pca_result[:, 1]

plt.figure(figsize=(10, 7))
sns.scatterplot(data=df_json_success, x='pca_one', y='pca_two', hue='kmeans_cluster', palette="viridis")
plt.title('PCA - Job Submission Clusters')
plt.show()


In [ ]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

def train_and_evaluate_xgboost(df, target, features, output_dir="/projectnb/peaclab-mon/boztop/resource-allocation/plots/"):
    os.makedirs(output_dir, exist_ok=True)
    
    colors = [
        'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white',
        'orange', 'purple', 'brown', 'pink', 'lime', 'teal', 'indigo', 'violet',
        '#FF6347', '#4682B4', '#32CD32', '#FFD700', '#6A5ACD', '#FF4500', '#DA70D6'
    ]
    
    total_underpredictions = 0
    total_overpredictions = 0
    total_samples = 0

    # Loop over each cluster
    for cluster_id in df['kmeans_cluster'].unique():
        print(f"Training XGBoost for Cluster {cluster_id}")
        
        cluster_data = df[df['kmeans_cluster'] == cluster_id]
        
        X = cluster_data[features]
        y = cluster_data[target]
        wallclock_requests = cluster_data['wallclock_req_seconds']
        
        X_train, X_test, y_train, y_test, wallclock_train, wallclock_test = train_test_split(
            X, y, wallclock_requests, test_size=0.2, random_state=42)
        
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)

        mse = mean_squared_error(y_test, y_pred)
        print(f"Cluster {cluster_id} - Mean Squared Error: {mse}")
        
        # Calculate underpredictions and overpredictions
        underpredictions = (y_pred < y_test).sum()
        underprediction_percentage = (underpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

        overpredictions = (y_pred > y_test).sum()
        overprediction_percentage = (overpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

        print(f"Cluster {cluster_id} - Number of underpredictions: {underpredictions} ({underprediction_percentage:.2f}%)")
        print(f"Cluster {cluster_id} - Number of overpredictions: {overpredictions} ({overprediction_percentage:.2f}%)")

        # Update totals
        total_underpredictions += underpredictions
        total_overpredictions += overpredictions
        total_samples += len(y_test)
        
        # Scatter plot of y_test vs y_pred
        plt.figure(figsize=(8, 6))
        plt.scatter(y_test, y_pred, color=np.random.choice(colors), label='Predicted vs Actual')
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Perfect prediction line')
        plt.xlabel('Actual Runtime')
        plt.ylabel('Predicted Runtime')
        plt.title(f'Cluster {cluster_id} - Predicted vs Actual Runtime')
        plt.legend()
        plt.grid(True)
        plot_filename = os.path.join(output_dir, f'cluster_{cluster_id}_runtime_prediction.png')
        plt.savefig(plot_filename)
        print(f"Figure saved for Cluster {cluster_id}: {plot_filename}")
        plt.show()

        # Wallclock comparison plot
        plt.figure(figsize=(8, 6))
        improvement = wallclock_test - y_pred  # Improvement = requested wallclock - predicted runtime
        plt.scatter(wallclock_test, improvement, color=np.random.choice(colors), label='Improvement')
        plt.axhline(y=0, color='red', linestyle='--', label='No Improvement')
        plt.xlabel('Requested Wallclock Time')
        plt.ylabel('Improvement (Wallclock - Predicted)')
        plt.title(f'Cluster {cluster_id} - Wallclock Time Improvement')
        plt.legend()
        plt.grid(True)
        plot_filename = os.path.join(output_dir, f'cluster_{cluster_id}_runtime_improvement.png')
        plt.savefig(plot_filename)
        print(f"Figure saved for Cluster {cluster_id}: {plot_filename}")
        plt.show()
    
    if total_samples > 0:
        total_underprediction_percentage = (total_underpredictions / total_samples) * 100
        total_overprediction_percentage = (total_overpredictions / total_samples) * 100
        print(f"\nTotal Number of underpredictions: {total_underpredictions} ({total_underprediction_percentage:.2f}%)")
        print(f"Total Number of overpredictions: {total_overpredictions} ({total_overprediction_percentage:.2f}%)")




In [ ]:
features = ['avg_power','processors_req', 'processors_used', 'nodes_req', 'nodes_used' ]
target = 'wallclock_used_seconds'

train_and_evaluate_xgboost(df_json_success, target, features)

In [ ]:
features = ['avg_power','processors_req', 'processors_used', 'nodes_req', 'nodes_used' ]
target = 'wallclock_used_seconds'

colors = [
    'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white',
    'orange', 'purple', 'brown', 'pink', 'lime', 'teal', 'indigo', 'violet',
    '#FF6347', '#4682B4', '#32CD32', '#FFD700', '#6A5ACD', '#FF4500', '#DA70D6'
]
    
output_dir = "/projectnb/peaclab-mon/boztop/resource-allocation/plots/"
os.makedirs(output_dir, exist_ok=True)

X = df_json_success[features]
y = df_json_success[target]
wallclock_requests = df_json_success['wallclock_req_seconds']
    
    
X_train, X_test, y_train, y_test, wallclock_train, wallclock_test = train_test_split(
    X, y, wallclock_requests, test_size=0.2, random_state=42)
    
    
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"NREL Eagle Total Dataset Mean Squared Error: {mse}")


underpredictions = (y_pred < y_test).sum()
underprediction_percentage = (underpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

overpredictions = (y_pred > y_test).sum()
overprediction_percentage = (overpredictions / len(y_test)) * 100 if len(y_test) > 0 else 0

print(f"NREL Eagle Total Dataset Number of underpredictions: {underpredictions} ({underprediction_percentage:.2f}%)")
print(f"NREL Eagle Total Dataset Number of overpredictions: {overpredictions} ({overprediction_percentage:.2f}%)")

    
    
# Scatter plot of y_test vs y_pred
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color=np.random.choice(colors), label='Predicted vs Actual')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--', label='Perfect prediction line')
plt.xlabel('Actual Runtime')
plt.ylabel('Predicted Runtime')
plt.title(f'NREL Eagle Total Dataset Predicted vs Actual Runtime')
plt.legend()
plt.grid(True)
plot_filename = os.path.join(output_dir, f'nrel_eagle_all_data_runtime_prediction.png')
plt.savefig(plot_filename)
plt.show()

# Wallclock comparison plot
plt.figure(figsize=(8, 6))
improvement = wallclock_test - y_pred  # Improvement = requested wallclock - predicted runtime
plt.scatter(wallclock_test, improvement, color=np.random.choice(colors), label='Improvement')
plt.axhline(y=0, color='red', linestyle='--', label='No Improvement')
plt.xlabel('Requested Wallclock Time')
plt.ylabel('Improvement (Wallclock - Predicted)')
plt.title(f'NREL Eagle Total Dataset Wallclock Time Improvement')
plt.legend()
plt.grid(True)
plot_filename = os.path.join(output_dir, f'nrel_eagle_all_data_runtime_improvement.png')
plt.savefig(plot_filename)
plt.show()
    